In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')
# Set Working Directory
%cd /content/drive/MyDrive/CSE6242

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [176]:
file_path = 'complete_file_merged_filtered.csv'
df = pd.read_csv(file_path)

In [158]:
df

,user_id,name_x,time,rating,text,gmap_id,name_y,address,latitude,longitude,avg_rating,num_of_reviews,predicted_label
0,1.115960e+20,Liema Hayes,1625931369083,5.0,Anytime I am downtown I have to get me some sp...,0x88f5047831dfc86d:0x985e3a6645f5ca4f,Noodle Cafe,"Noodle Cafe, 233 Peachtree St # B70, Atlanta, ...",33.760404,-84.386319,4.2,35,0
1,1.087151e+20,Evan Guthrie,1614374734371,5.0,Noodle Cafe at Peachtree Center in Atlanta is ...,0x88f5047831dfc86d:0x985e3a6645f5ca4f,Noodle Cafe,"Noodle Cafe, 233 Peachtree St # B70, Atlanta, ...",33.760404,-84.386319,4.2,35,0
2,1.163122e+20,Jerome Narain,1612374257054,3.0,Updated Menu per Covid-19.,0x88f5047831dfc86d:0x985e3a6645f5ca4f,Noodle Cafe,"Noodle Cafe, 233 Peachtree St # B70, Atlanta, ...",33.760404,-84.386319,4.2,35,0
3,1.080704e+20,osheona r,1571850144379,3.0,Food used to be much better. I came in today a...,0x88f5047831dfc86d:0x985e3a6645f5ca4f,Noodle Cafe,"Noodle Cafe, 233 Peachtree St # B70, Atlanta, ...",33.760404,-84.386319,4.2,35,0
4,1.095364e+20,Matthew Wamboldt,1596820182685,5.0,"Timely service and decent, affordable Thai/Chi...",0x88f5047831dfc86d:0x985e3a6645f5ca4f,Noodle Cafe,"Noodle Cafe, 233 Peachtree St # B70, Atlanta, ...",33.760404,-84.386319,4.2,35,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
672171,1.092219e+20,arda konik,1510058390462,3.0,NaN,0x88f505fd4257da21:0x6141c39ce5d8d43e,Truva,"Truva, 60 Andrew Young International Blvd NE, ...",33.759901,-84.385785,4.5,672,0
672172,1.132573e+20,negar rahmatpour,1509329351562,3.0,NaN,0x88f505fd4257da21:0x6141c39ce5d8d43e,Truva,"Truva, 60 Andrew Young International Blvd NE, ...",33.759901,-84.385785,4.5,672,0
672173,1.146886e+20,Andre Rocha,1461450767619,5.0,NaN,0x88f505fd4257da21:0x6141c39ce5d8d43e,Truva,"Truva, 60 Andrew Young International Blvd NE, ...",33.759901,-84.385785,4.5,672,0
672174,1.049787e+20,Motasem .magairah,1424489603604,5.0,NaN,0x88f505fd4257da21:0x6141c39ce5d8d43e,Truva,"Truva, 60 Andrew Young International Blvd NE, ...",33.759901,-84.385785,4.5,672,0


In [184]:
# Convert 'time' column to datetime
df['time'] = pd.to_datetime(df['time'], unit='ms')

# Current time for decay calculation
current_time = datetime.now()

# Implementing the time-decay model with more emphasis on recent reviews
def adjusted_time_decay_rating(days_since_review):
    # Boost more for recent reviews (last 90 days)
    if days_since_review <= 90:
        return 1.2  # Adjust this factor as needed to boost recent reviews

    # Apply steeper decay for older reviews
    else:
        life_cycle = 365  # Shorter life cycle for quicker decay
        decay_rate = 0.90  # Steeper decay
        return decay_rate ** (days_since_review / life_cycle)

# Apply the time decay to ratings
df['days_since_review'] = (current_time - df['time']).dt.days
df['decay_factor'] = df['days_since_review'].apply(adjusted_time_decay_rating)

# Normalize weights for each restaurant
df['normalized_weight'] = df.groupby('gmap_id')['decay_factor'].transform(lambda x: x / x.sum())

# Calculate the weighted average
df['weighted_rating'] = df['rating'] * df['normalized_weight']
weighted_avg_rating = df.groupby('gmap_id')['weighted_rating'].sum().reset_index(name='weighted_avg_rating')

# Set gmap_id as the index for merging
df.set_index('gmap_id', inplace=True)
weighted_avg_rating.set_index('gmap_id', inplace=True)

# Merge the weighted average rating into the original dataset
df = df.merge(weighted_avg_rating, on='gmap_id', how='left')

# Reset the index
df.reset_index(inplace=True)

In [160]:
df

,gmap_id,user_id,name_x,time,rating,text,name_y,address,latitude,longitude,avg_rating,num_of_reviews,predicted_label,days_since_review,decay_factor,normalized_weight,weighted_rating,weighted_avg_rating
0,0x88f5047831dfc86d:0x985e3a6645f5ca4f,1.115960e+20,Liema Hayes,2021-07-10 15:36:09.083,5.0,Anytime I am downtown I have to get me some sp...,Noodle Cafe,"Noodle Cafe, 233 Peachtree St # B70, Atlanta, ...",33.760404,-84.386319,4.2,35,0,868,0.882252,0.037453,0.187265,4.269167
1,0x88f5047831dfc86d:0x985e3a6645f5ca4f,1.087151e+20,Evan Guthrie,2021-02-26 21:25:34.371,5.0,Noodle Cafe at Peachtree Center in Atlanta is ...,Noodle Cafe,"Noodle Cafe, 233 Peachtree St # B70, Atlanta, ...",33.760404,-84.386319,4.2,35,0,1001,0.865478,0.036741,0.183705,4.269167
2,0x88f5047831dfc86d:0x985e3a6645f5ca4f,1.163122e+20,Jerome Narain,2021-02-03 17:44:17.054,3.0,Updated Menu per Covid-19.,Noodle Cafe,"Noodle Cafe, 233 Peachtree St # B70, Atlanta, ...",33.760404,-84.386319,4.2,35,0,1025,0.862485,0.036614,0.109842,4.269167
3,0x88f5047831dfc86d:0x985e3a6645f5ca4f,1.080704e+20,osheona r,2019-10-23 17:02:24.379,3.0,Food used to be much better. I came in today a...,Noodle Cafe,"Noodle Cafe, 233 Peachtree St # B70, Atlanta, ...",33.760404,-84.386319,4.2,35,0,1494,0.806035,0.034217,0.102652,4.269167
4,0x88f5047831dfc86d:0x985e3a6645f5ca4f,1.095364e+20,Matthew Wamboldt,2020-08-07 17:09:42.685,5.0,"Timely service and decent, affordable Thai/Chi...",Noodle Cafe,"Noodle Cafe, 233 Peachtree St # B70, Atlanta, ...",33.760404,-84.386319,4.2,35,0,1205,0.840367,0.035675,0.178375,4.269167
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
672171,0x88f505fd4257da21:0x6141c39ce5d8d43e,1.092219e+20,arda konik,2017-11-07 12:39:50.462,3.0,NaN,Truva,"Truva, 60 Andrew Young International Blvd NE, ...",33.759901,-84.385785,4.5,672,0,2209,0.727004,0.001573,0.004718,4.467696
672172,0x88f505fd4257da21:0x6141c39ce5d8d43e,1.132573e+20,negar rahmatpour,2017-10-30 02:09:11.562,3.0,NaN,Truva,"Truva, 60 Andrew Young International Blvd NE, ...",33.759901,-84.385785,4.5,672,0,2217,0.726165,0.001571,0.004712,4.467696
672173,0x88f505fd4257da21:0x6141c39ce5d8d43e,1.146886e+20,Andre Rocha,2016-04-23 22:32:47.619,5.0,NaN,Truva,"Truva, 60 Andrew Young International Blvd NE, ...",33.759901,-84.385785,4.5,672,0,2771,0.670362,0.001450,0.007250,4.467696
672174,0x88f505fd4257da21:0x6141c39ce5d8d43e,1.049787e+20,Motasem .magairah,2015-02-21 03:33:23.604,5.0,NaN,Truva,"Truva, 60 Andrew Young International Blvd NE, ...",33.759901,-84.385785,4.5,672,0,3199,0.630205,0.001363,0.006816,4.467696


In [178]:
# Calculate the global weighted average rating
global_weighted_avg = df['weighted_avg_rating'].mean()

# Count the number of reviews for each gmap_id
rating_counts = df.groupby('gmap_id').size()

# Calculate the average number of reviews per gmap_id (C)
# C = rating_counts.mean()
C = rating_counts.median()
# C = 5

# Function to calculate Bayesian average
def bayesian_average(ratings, C=C, m=global_weighted_avg):
    R = ratings.mean()  # mean rating for this gmap_id
    v = len(ratings)    # number of reviews for this gmap_id
    return (v / (v + C)) * R + (C / (v + C)) * m

# Calculate the Bayesian average for each gmap_id based on weighted_avg_rating
bayesian_avg_rating = df.groupby('gmap_id')['weighted_avg_rating'].apply(bayesian_average).reset_index(name='bayesian_avg_rating')

# Create a dictionary for mapping gmap_id to bayesian_avg_rating
bayesian_avg_map = dict(zip(bayesian_avg_rating['gmap_id'], bayesian_avg_rating['bayesian_avg_rating']))

# Map the bayesian_avg_rating to each row in the original DataFrame
df['updated_weighted_avg_rating'] = df['gmap_id'].map(bayesian_avg_map)


In [95]:
D= rating_counts.median()
D

272.5

In [162]:
df

,gmap_id,user_id,name_x,time,rating,text,name_y,address,latitude,longitude,avg_rating,num_of_reviews,predicted_label,days_since_review,decay_factor,normalized_weight,weighted_rating,weighted_avg_rating,updated_weighted_avg_rating
0,0x88f5047831dfc86d:0x985e3a6645f5ca4f,1.115960e+20,Liema Hayes,2021-07-10 15:36:09.083,5.0,Anytime I am downtown I have to get me some sp...,Noodle Cafe,"Noodle Cafe, 233 Peachtree St # B70, Atlanta, ...",33.760404,-84.386319,4.2,35,0,868,0.882252,0.037453,0.187265,4.269167,4.156541
1,0x88f5047831dfc86d:0x985e3a6645f5ca4f,1.087151e+20,Evan Guthrie,2021-02-26 21:25:34.371,5.0,Noodle Cafe at Peachtree Center in Atlanta is ...,Noodle Cafe,"Noodle Cafe, 233 Peachtree St # B70, Atlanta, ...",33.760404,-84.386319,4.2,35,0,1001,0.865478,0.036741,0.183705,4.269167,4.156541
2,0x88f5047831dfc86d:0x985e3a6645f5ca4f,1.163122e+20,Jerome Narain,2021-02-03 17:44:17.054,3.0,Updated Menu per Covid-19.,Noodle Cafe,"Noodle Cafe, 233 Peachtree St # B70, Atlanta, ...",33.760404,-84.386319,4.2,35,0,1025,0.862485,0.036614,0.109842,4.269167,4.156541
3,0x88f5047831dfc86d:0x985e3a6645f5ca4f,1.080704e+20,osheona r,2019-10-23 17:02:24.379,3.0,Food used to be much better. I came in today a...,Noodle Cafe,"Noodle Cafe, 233 Peachtree St # B70, Atlanta, ...",33.760404,-84.386319,4.2,35,0,1494,0.806035,0.034217,0.102652,4.269167,4.156541
4,0x88f5047831dfc86d:0x985e3a6645f5ca4f,1.095364e+20,Matthew Wamboldt,2020-08-07 17:09:42.685,5.0,"Timely service and decent, affordable Thai/Chi...",Noodle Cafe,"Noodle Cafe, 233 Peachtree St # B70, Atlanta, ...",33.760404,-84.386319,4.2,35,0,1205,0.840367,0.035675,0.178375,4.269167,4.156541
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
672171,0x88f505fd4257da21:0x6141c39ce5d8d43e,1.092219e+20,arda konik,2017-11-07 12:39:50.462,3.0,NaN,Truva,"Truva, 60 Andrew Young International Blvd NE, ...",33.759901,-84.385785,4.5,672,0,2209,0.727004,0.001573,0.004718,4.467696,4.367320
672172,0x88f505fd4257da21:0x6141c39ce5d8d43e,1.132573e+20,negar rahmatpour,2017-10-30 02:09:11.562,3.0,NaN,Truva,"Truva, 60 Andrew Young International Blvd NE, ...",33.759901,-84.385785,4.5,672,0,2217,0.726165,0.001571,0.004712,4.467696,4.367320
672173,0x88f505fd4257da21:0x6141c39ce5d8d43e,1.146886e+20,Andre Rocha,2016-04-23 22:32:47.619,5.0,NaN,Truva,"Truva, 60 Andrew Young International Blvd NE, ...",33.759901,-84.385785,4.5,672,0,2771,0.670362,0.001450,0.007250,4.467696,4.367320
672174,0x88f505fd4257da21:0x6141c39ce5d8d43e,1.049787e+20,Motasem .magairah,2015-02-21 03:33:23.604,5.0,NaN,Truva,"Truva, 60 Andrew Young International Blvd NE, ...",33.759901,-84.385785,4.5,672,0,3199,0.630205,0.001363,0.006816,4.467696,4.367320


In [179]:
# Define a function for min-max normalization within each group
def normalize_within_group(df, rating_column):
    min_rating = df[rating_column].min()
    max_rating = df[rating_column].max()
    df['normalized_weighted_rating'] = 5 * (df[rating_column] - min_rating) / (max_rating - min_rating)
    return df

# Apply the normalization function to each gmap_id group
df = df.groupby('gmap_id', group_keys=False).apply(normalize_within_group, rating_column='weighted_rating')

# Round the normalized ratings to one decimal place
df['normalized_weighted_rating'] = df['normalized_weighted_rating'].round(1)

In [180]:
# Round weighted_rating and weighted_avg_rating columns to one decimal place
# df['normalized_weighted_rating'] = df['normalized_weighted_rating'].round(0)
df['weighted_avg_rating'] = df['weighted_avg_rating'].round(1)
df['updated_weighted_avg_rating'] = df['updated_weighted_avg_rating'].round(1)

In [181]:
df['weighted_rating']

0         0.171456
1         0.170145
2         0.101945
3         0.099222
4         0.168152
            ...   
672171    0.004854
672172    0.004852
672173    0.007832
672174    0.007641
672175    0.008379
Name: weighted_rating, Length: 672176, dtype: float64

In [182]:
# Convert the 'time' column to datetime objects
df['time'] = pd.to_datetime(df['time'])

# Convert the datetime objects to Unix timestamp in milliseconds
df['time'] = (df['time'] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1ms')

In [183]:
df.to_csv('complete_file_merged_filtered_adjusted_1125.csv', index=False)

In [24]:
df_1124 = pd.read_csv('complete_file_merged_filtered_adjusted_1125.csv')

In [125]:
example = df_1124[df_1124['gmap_id']=='0x88f50464ee0c6559:0xe2fa0e6431e3fdb8']
example

,gmap_id,user_id,name_x,time,rating,text,name_y,address,latitude,longitude,avg_rating,num_of_reviews,predicted_label,days_since_review,decay_factor,normalized_weight,weighted_rating,weighted_avg_rating,updated_weighted_avg_rating,normalized_weighted_rating
110111,0x88f50464ee0c6559:0xe2fa0e6431e3fdb8,1.097890e+20,Eric Larsen,1618622,3.0,There is a lot of hype around this place but i...,The Varsity,"The Varsity, 61 North Avenue NW, Atlanta, GA 3...",33.771595,-84.389303,4.1,9998,0,951,0.759941,0.000165,0.000494,4.0,4.0,2.8
110112,0x88f50464ee0c6559:0xe2fa0e6431e3fdb8,1.036016e+20,Malaya Zaire,1616332,5.0,The fries and the hot dogs are so good and fre...,The Varsity,"The Varsity, 61 North Avenue NW, Atlanta, GA 3...",33.771595,-84.389303,4.1,9998,0,978,0.754041,0.000164,0.000818,4.0,4.0,4.9
110113,0x88f50464ee0c6559:0xe2fa0e6431e3fdb8,1.043905e+20,Matthew Reynolds,1616286,5.0,AMAZING FOOD and GREAT RESTAURANT TO EAT AT!\n...,The Varsity,"The Varsity, 61 North Avenue NW, Atlanta, GA 3...",33.771595,-84.389303,4.1,9998,0,978,0.754041,0.000164,0.000818,4.0,4.0,4.9
110114,0x88f50464ee0c6559:0xe2fa0e6431e3fdb8,1.072065e+20,Leon Fot,1615751,5.0,"Absolutely delicious, good, super quick servic...",The Varsity,"The Varsity, 61 North Avenue NW, Atlanta, GA 3...",33.771595,-84.389303,4.1,9998,0,985,0.752519,0.000163,0.000816,4.0,4.0,4.9
110115,0x88f50464ee0c6559:0xe2fa0e6431e3fdb8,1.011586e+20,Terry Chapman,1607229,5.0,I always love The Varsity. It's an old histori...,The Varsity,"The Varsity, 61 North Avenue NW, Atlanta, GA 3...",33.771595,-84.389303,4.1,9998,0,1083,0.731530,0.000159,0.000793,4.0,4.0,4.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117960,0x88f50464ee0c6559:0xe2fa0e6431e3fdb8,1.032185e+20,Marge Anderson,1547273,4.0,NaN,The Varsity,"The Varsity, 61 North Avenue NW, Atlanta, GA 3...",33.771595,-84.389303,4.1,9998,0,1777,0.598729,0.000130,0.000519,4.0,4.0,3.0
117961,0x88f50464ee0c6559:0xe2fa0e6431e3fdb8,1.028527e+20,Brittany Bones,1515289,5.0,NaN,The Varsity,"The Varsity, 61 North Avenue NW, Atlanta, GA 3...",33.771595,-84.389303,4.1,9998,0,2147,0.538079,0.000117,0.000583,4.0,4.0,3.4
117962,0x88f50464ee0c6559:0xe2fa0e6431e3fdb8,1.048725e+20,Robert Thomas,1536889,5.0,NaN,The Varsity,"The Varsity, 61 North Avenue NW, Atlanta, GA 3...",33.771595,-84.389303,4.1,9998,0,1897,0.578344,0.000125,0.000627,4.0,4.0,3.7
117963,0x88f50464ee0c6559:0xe2fa0e6431e3fdb8,1.088036e+20,Daryl Carter,1513278,5.0,NaN,The Varsity,"The Varsity, 61 North Avenue NW, Atlanta, GA 3...",33.771595,-84.389303,4.1,9998,0,2171,0.534364,0.000116,0.000579,4.0,4.0,3.4


In [130]:
# To select the first row (at index 0) and only specific columns by index position:
example_row = df_1124.iloc[117962, [6, 4, 15, 16, 19]]  # Here 1, 2 are the integer positions of the desired columns
example_row


name_y                        The Varsity
rating                                5.0
normalized_weight                0.000125
weighted_rating                  0.000627
normalized_weighted_rating            3.7
Name: 117962, dtype: object